# XCS in Multiplexer

In [1]:
# import logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

import gym
import gym_multiplexer  

from lcs.agents import EnvironmentAdapter
from lcs.agents.xcs import XCS, Configuration

# Environment

In [2]:
mpx = gym.make('boolean-multiplexer-6bit-v0')

Example of return value

In [3]:
state = mpx.reset()

The last bit is changed after executing an action. It if was valid it's set to `1`, otherwise `0`. This behavior is not needed for the XCS, therefore signal can becompacted

## Agent

In [4]:
class MultiplexerAdapter(EnvironmentAdapter):
    @classmethod
    def to_genotype(cls, env_state):
        return list(map(str, map(int, env_state[:-1])))  # skip last bit

def mpx_metrics(xcs: XCS, environment):
    return {
        'population': len(xcs.population),
        'numerosity': sum(cl.numerosity for cl in xcs.population)
    }
    
cfg = Configuration(number_of_actions=2,
                    max_population=1000,
                    gamma=0.9,
                    # ga_threshold = 9999999999999999,
                    # covering_wildcard_chance=1,
                    metrics_trial_frequency=100,
                    # mutation_chance=0,
                    environment_adapter=MultiplexerAdapter(),
                    user_metrics_collector_fcn=mpx_metrics)

agent = XCS(cfg)

## Experiments

In [5]:
%%time
explore_population, explore_metrics = agent.explore(mpx, 10_000, decay=False)

Wall time: 3.98 s


In [6]:
print(f'Explore population count: {len(explore_population)}\n')

print('Top 15 classifiers:')
for cl in sorted(explore_population, key=lambda cl: -cl.fitness)[15:]:
    print(f'{cl.condition}\t{cl.action}')

Explore population count: 40

Top 15 classifiers:
#01110	1
#01111	0
#####0	0
#011#0	1
##0001	1
0##011	0
#1#10#	0
###1##	0
1#00#1	0
###11#	0
0###10	1
11##01	0
0##011	1
1#00#1	1
000#0#	0
#10#1#	1
######	0
#11#10	1
0###10	0
1#00#0	1
00#0##	0
#0##10	1
01###1	0
##01##	0
#0#1##	1


See if there are some duplicate classifiers.

In [7]:
rules = [(cl.condition, cl.action) for cl in explore_population]
len(set(rules))

40

There are duplicate classifiers found.

In [8]:
for rule in explore_population:
    for second_rule in explore_population:
        if rule.condition == second_rule.condition and rule.action == second_rule.action and id(rule) != id(second_rule):
            print("Duplicate Classifier found:")
            print(rule)
            print(second_rule)

In [9]:
for rule in explore_population:
    print(rule)

Cond:01###1 - Act:0 - Num:2 [fit: 0.000, exp: 198.00, pred: 521.742]
Cond:#01111 - Act:0 - Num:27 [fit: 0.000, exp: 111.00, pred: 476.774]
Cond:00#0## - Act:0 - Num:6 [fit: 0.000, exp: 208.00, pred: 521.456]
Cond:1#00#0 - Act:1 - Num:11 [fit: 0.000, exp: 165.00, pred: 521.855]
Cond:0###10 - Act:0 - Num:11 [fit: 0.000, exp: 173.00, pred: 485.328]
Cond:0###10 - Act:1 - Num:35 [fit: 0.000, exp: 148.00, pred: 453.594]
Cond:1#00#1 - Act:1 - Num:24 [fit: 0.000, exp: 175.00, pred: 530.429]
Cond:0#1#11 - Act:1 - Num:55 [fit: 0.000, exp: 119.00, pred: 503.860]
Cond:#011#0 - Act:1 - Num:21 [fit: 0.000, exp: 102.00, pred: 574.861]
Cond:#11#10 - Act:1 - Num:19 [fit: 0.000, exp: 187.00, pred: 501.388]
Cond:1#00#1 - Act:0 - Num:32 [fit: 0.000, exp: 152.00, pred: 412.653]
Cond:#11111 - Act:0 - Num:38 [fit: 0.000, exp: 84.00, pred: 520.065]
Cond:0##011 - Act:0 - Num:60 [fit: 0.000, exp: 157.00, pred: 528.572]
Cond:0##011 - Act:1 - Num:32 [fit: 0.000, exp: 158.00, pred: 483.638]
Cond:11##01 - Act:0 - N

In [10]:
detected_duplicates = 0
for i in range(100):
    agent = XCS(cfg)
    explore_population, explore_metrics = agent.explore(mpx, 10_000, decay=False)
    rules = [(cl.condition, cl.action) for cl in explore_population]
    if len(explore_population) > len(set(rules)):
        print(f"Experiment {i}: Duplicate found: {len(explore_population)}, {len(set(rules))}")
        detected_duplicates += 1
    else:
        print(f"Experiment {i}: No duplicates")
print(f"{detected_duplicates / 100}")

Experiment 0: No duplicates
Experiment 1: Duplicate found: 44, 42
Experiment 2: No duplicates
Experiment 3: Duplicate found: 36, 35
Experiment 4: No duplicates
Experiment 5: Duplicate found: 39, 37
Experiment 6: Duplicate found: 46, 45
Experiment 7: No duplicates
Experiment 8: Duplicate found: 38, 37
Experiment 9: No duplicates
Experiment 10: No duplicates
Experiment 11: Duplicate found: 36, 35
Experiment 12: Duplicate found: 34, 33
Experiment 13: Duplicate found: 47, 45
Experiment 14: Duplicate found: 31, 30
Experiment 15: No duplicates
Experiment 16: Duplicate found: 31, 30
Experiment 17: No duplicates
Experiment 18: No duplicates
Experiment 19: Duplicate found: 44, 43
Experiment 20: No duplicates
Experiment 21: Duplicate found: 38, 37
Experiment 22: No duplicates
Experiment 23: No duplicates
Experiment 24: No duplicates
Experiment 25: Duplicate found: 48, 46
Experiment 26: No duplicates
Experiment 27: No duplicates
Experiment 28: Duplicate found: 42, 40
Experiment 29: Duplicate foun